<a href="https://colab.research.google.com/github/jdKheppar/Chatbot-Project/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')#Sentence tokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('./intents.json').read() # read json file
intents = json.loads(data_file) # load json file

In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)# add each elements into list
        #combination between patterns and intents
        documents.append((w, intent['tag']))#add single element into end of list
        # add to tag in our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
nltk.download('wordnet') #lexical database for the English language

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents\n", documents, "\n")
# classes = intents[tag]
print (len(classes), "classes\n", classes, "\n")
# words = all words, vocabulary
print (len(words), "unique lemmatized words\n", words, "\n")
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

422 documents
 [(['Hi'], 'greeting'), (['How', 'are', 'you', '?'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['What', "'s", 'up'], 'greeting'), (['how', 'are', 'ya'], 'greeting'), (['heyy'], 'greeting'), (['whatsup'], 'greeting'), (['?', '?', '?', '?', '?', '?', '?', '?'], 'greeting'), (['cya'], 'goodbye'), (['see', 'you'], 'goodbye'), (['bye', 'bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Bye'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['talk', 'to', 'you', 'later'], 'goodbye'), (['ttyl'], 'goodbye'), (['i', 'got', 'to', 'go'], 'goodbye'), (['gtg'], 'goodbye'), (['what', 'is', 'the', 'name', 'of', 'your', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'your', 'creators'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'creato

In [ ]:
# create our training data
from sklearn.model_selection import train_test_split
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words
    pattern_words = doc[0]
    # convert pattern_words in lower case
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create bag of words array,if word match found in current pattern then put 1 otherwise 0.[row * colm(263)]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # in output array 0 value for each tag ang 1 value for matched tag.[row * colm(8)]
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle training and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test. X - patterns(words), Y - intents(tags)
train_data, test_data = train_test_split(training, test_size=0.2, random_state=42)

test_x = list(test_data[:,0])
test_y = list(test_data[:,1])
train_x = list(train_data[:,0])
train_y = list(train_data[:,1])
print("Training data created")

Training data created


In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [ ]:
# Create model - 3 layers. First layer 64 neurons, second layer 32 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(64, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(train_y[0]), activation='softmax'))
print("First layer:",model.layers[0].get_weights()[0])

First layer: [[ 0.10457352  0.04242885 -0.10423896 ...  0.0268044   0.06624343
   0.07299136]
 [-0.04119976 -0.07166049 -0.03316431 ...  0.02648526  0.09588648
   0.00958806]
 [ 0.00772668  0.11682083  0.09553756 ... -0.04984259  0.06700695
   0.00562441]
 ...
 [-0.05818146  0.09655553 -0.03840631 ... -0.08837566 -0.11657199
  -0.017756  ]
 [ 0.04796933 -0.12336817  0.10174611 ...  0.01113139 -0.04244635
   0.08307426]
 [-0.11301258 -0.0289164  -0.09439796 ... -0.03828824  0.06936562
   0.03578992]]


In [ ]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
68/68 [==============================] - 9s 5ms/step - loss: 3.7227 - accuracy: 0.0564
Epoch 2/200
68/68 [==============================] - 0s 5ms/step - loss: 3.5403 - accuracy: 0.2077
Epoch 3/200
68/68 [==============================] - 0s 5ms/step - loss: 3.2079 - accuracy: 0.2136
Epoch 4/200
68/68 [==============================] - 0s 5ms/step - loss: 2.8071 - accuracy: 0.2404
Epoch 5/200
68/68 [==============================] - 0s 6ms/step - loss: 2.4058 - accuracy: 0.4125
Epoch 6/200
68/68 [==============================] - 0s 5ms/step - loss: 1.9931 - accuracy: 0.5994
Epoch 7/200
68/68 [==============================] - 0s 7ms/step - loss: 1.5978 - accuracy: 0.7240
Epoch 8/200
68/68 [==============================] - 0s 5ms/step - loss: 1.2486 - accuracy: 0.8042
Epoch 9/200
68/68 [==============================] - 0s 6ms/step - loss: 0.9627 - accuracy: 0.8635
Epoch 10/200
68/68 [==============================] - 0s 5ms/step - loss: 0.7399 - accuracy: 0.9080
Epoch 11/

In [ ]:
from keras.models import load_model

loaded_model = load_model('./chatbot_model.h5')

intents = json.loads(open('./intents.json').read())

words = pickle.load(open('./words.pkl','rb'))

classes = pickle.load(open('./classes.pkl','rb'))



In [ ]:
# Predict and evaluate accuracy
# Use the Keras evaluate() method to get the accuracy
loss, accuracy = loaded_model.evaluate(np.array(test_x), np.array(test_y), verbose=0)
print(f'Test accuracy: {accuracy*100:.3f}')
print(f'Test loss: {loss:.3f}')

Test accuracy: 78.824
Test loss: 1.695


In [ ]:
def clean_up_sentence(sentence):
  # tokenize the pattern - split words into array

  sentence_words = nltk.word_tokenize(sentence)
  #print(sentence_words)
  # stem each word - create short form for word

  sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
  #print(sentence_words)

  return sentence_words

In [ ]:
def bow(sentence, words, show_details=True):
  # tokenize the pattern

  sentence_words = clean_up_sentence(sentence)
  #print(sentence_words)

  # bag of words - matrix of N words, vocabulary matrix

  bag = [0]*len(words) 
  #print(bag)

  for s in sentence_words:  
      for i,w in enumerate(words):
          if w == s: 
              # assign 1 if current word is in the vocabulary position
              bag[i] = 1
              if show_details:
                  print ("found in bag: %s" % w)
              #print ("found in bag: %s" % w)
  #print(bag)
  return(np.array(bag))              

In [ ]:
def predict_class(sentence, model):
  # filter out predictions below a threshold

  p = bow(sentence, words,show_details=False)
  #print(p)

  res = loaded_model.predict(np.array([p]))[0]
  #print(res)

  ERROR_THRESHOLD = 0.25

  results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
  #print(results)
  # sort by strength of probability

  results.sort(key=lambda x: x[1], reverse=True)
  #print(results)
  return_list = []

  for r in results:
      return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

  return return_list
  #print(return_list)

In [ ]:
def getResponse(ints, intents_json):
  tag = ints[0]['intent']
  #print(tag)

  list_of_intents = intents_json['intents']
  #print(list_of_intents)
  for i in list_of_intents:
      if(i['tag']== tag):
          result = random.choice(i['responses'])
          break  
  return result

In [ ]:
def chatbot_response(text):
   ints = predict_class(text, model)
   #print(ints)

   res = getResponse(ints, intents)
   #print(res)
   return res

In [ ]:
start = True
while start:
  query = input('Enter Message:')
  if query in ['quit','exit','bye']:
      start = False
      continue
  try:
      res = chatbot_response(query)
      print(res)
  except:
      print('You may need to rephrase your question.')

Enter Message:hello
1/1 [==============================] - 0s 117ms/step
Hi there, how can I help?
Enter Message:tell about fees
1/1 [==============================] - 0s 18ms/step
For Fee detail visit <a target="_blank" href="LINK"> here</a>
Enter Message:about admission
1/1 [==============================] - 0s 17ms/step
Application can also be submitted online through the Unversity's  <a target="_blank" href="LINK OF ADMISSION DOCUMENT">website</a>
